### Why a Well-Designed State Matters
Think of the State object as the single source of truth for your agent's entire operation. At any point in the graph's execution, the state tells you everything you need to know:

What was the original question?

What is the history of the conversation?

What tools has the agent called?

What were the results of those tool calls?

Has the agent tried to re-plan? How many times?

By structuring this information cleanly in a TypedDict, we make our nodes simpler to write and the entire graph's flow much easier to follow and debug.

In [ ]:
import os
from dotenv import load_dotenv
load_dotenv()
google_api_key = os.getenv("GOOGLE_API_KEY")

2 ways to update the memory:<br>
Overwrite<br>
append

In [2]:
from typing import TypedDict

class BasicState(TypedDict):
    question: str
    answer: str

def generate_answer(state:BasicState):
    print(f"Answering: {state['question']}")
    return {'answer':"answer is ME"}
# If the initial state is {"question": "What is the meaning of life?", "answer": ""}
# After running the node, the state becomes:
# {"question": "What is the meaning of life?", "answer": "The answer is 42."}
# The empty string was overwritten.

2. Modify/Append (Using Annotated)<br>
Often, we don't want to replace data; we want to add to it. This is where ***Annotated*** comes in. It lets you attach a reducer function to a key, which tells LangGraph how to merge the new value with the old one.

In [9]:
import operator
from typing import Annotated

class AgentState(TypedDict):
    tool_calls:Annotated[int, operator.add]
def call_tool(state:AgentState):
    print("calling the tool")
    # LangGraph will see the Annotated reducer and do:
    # new_state['tool_calls'] = operator.add(old_state['tool_calls'], 1)
    return{"tools_calls":1}

### Complex Schema
state for a research agent

In [10]:
from typing import TypedDict, Annotated, List
from langchain_core.messages import AnyMessage
from langgraph.graph.message import add_messages
import operator

class ResearchAgentState(TypedDict):
    task:str
    messages:Annotated[list[AnyMessage], add_messages]
    documents:List[str]
    replan_count:Annotated[int, operator.add]
#Your State TypedDict is the blueprint for your agent's memory. Design it thoughtfully.

## If the State is the agent's memory and workbench, Nodes are the individual tools and workers that operate on it <br>
LangGraph's design is that a node is nothing more than a standard Python function (or any callable) with a specific, simple signature.

**The Node Signature: Input and Output**<br>
Every function you want to use as a node must follow a simple contract:
<ol>
<li>Input: It must accept a single argument, which will be the current state object (an instance of your TypedDict).</li>

<li>Output: It must return a dictionary. The keys of this dictionary must be a subset of the keys defined in your State TypedDict. The values will be used to update the state according to the rules we defined in the last lesson (overwrite or append/modify).</li></ol>

In [ ]:
class ResearchAgentState(TypedDict):
    task: str
    messages: Annotated[list[AnyMessage], add_messages]
    documents: List[str]
    replan_count: Annotated[int, operator.add]

def my_node_function(state: ResearchAgentState):
    current_task = state['task']
    current_messages= state['messages']
    
    print("perfonming REsearch.....")
    new_documents = ["doc1", "doc2"]  # Simulate some document retrieval
    return {"documents": new_documents} #overwrite the documents key

Best Practice: Single-Responsibility Nodes
While you could write a single, massive node that calls an LLM, then parses its output, then decides which tool to use, and then executes it, this is a bad practice. It makes your graph monolithic, hard to debug, and difficult to reuse.

The power of the graph architecture is realized when you break down your logic into small, modular, single-responsibility nodes.

In [ ]:
from langchain_google_genai import GoogleGenerativeAI
from langchain_core.messages.tool import ToolMessage
from langchain_core.messages import SystemMessage

model= GoogleGenerativeAI(model="gemini-2.0-flash",google_api_key=google_api_key)

def call_planner(state: ResearchAgentState):
    """Decide the next step based on the current state."""
    #uses messages history to call llm
    print("Planning next steps...")
    response = model.invoke(state['messages'])
    return {"messages": [response]}  

def execute_search(state: ResearchAgentState):
    """Perform a search based on the current task."""
    print("Executing search...")
    ai_message=state['messages'][-1]
    
    search_query= "LangGraph"
    
    result = f"this is a search result for {search_query}"
    
    # The result is returned as a ToolMessage to be added to the history
    tool_message = ToolMessage(content=result, tool_call_id=ai_message.tool_calls[0]['id']) # A real ID would be here
    return {"documents":[result], "messages": [tool_message]}

def should_replan(state: ResearchAgentState) -> dict:
    print("---RE-PLANNING---")
    # This node is called when the agent decides its first plan wasn't good enough.
    replan_message = SystemMessage(content="The previous plan was not sufficient. I will try again.")
    return {
        "messages": [replan_message],
        "replan_count": 1 # This will be added to the existing count by operator.add
    }

## Conditional Edges for Intelligent Routing
We have our State (the memory) and our Nodes (the workers). Now we need the manager—the logic that directs the flow of work from one node to another. This is the role of Edges.<br>

<ol>
<li>Standard Edges (add_edge): This is a simple, unconditional connection. It says, "After node A finishes, always go to node B." This is useful for linear sequences within your graph.</li>
<li>Conditional Edges (add_conditional_edges): This is the decision-making mechanism. It says, "After node A finishes, run a special routing function. This function will inspect the state and decide whether to go to node B, node C, or even end the process."</li>
</ol>

The routing function has the same signature as a node (it takes the state as input), but instead of returning a dictionary to update the state, it returns a string. This string is the name of the next node to execute.

lets implement the call_planner node:<br>
<ul><li>if llm's response contains req to use tool-> go to execute_search</li>
<li>if the LLM response is a final answer we should finish the graph</li></ul>


In [15]:
from langchain_core.messages import AIMessage

def route_after_planner(state:ResearchAgentState) ->str:
    """
    Inspects the last message in the state and decides where to go next.

    Returns:
        A string that is the name of the next node.
    """
    print("Routing after planner...")
    last_message = state['messages'][-1]
    if isinstance(last_message, AIMessage):
        print("AI wants to call a tool")
        return "execute_search"
    else:
        print("AI has finished its work")
        return "end"
    

## Graph

In [ ]:
from langgraph.graph import StateGraph, END

graph_builder = StateGraph(ResearchAgentState)

graph_builder.add_node("planner", call_planner)

graph_builder.add_node("execute_search", execute_search)

graph_builder.set_entry_point("planner")

graph_builder.add_conditional_edges(
    "planner", #decision is made after planner runs
    
    route_after_planner, #this function decides the next node
    {
        "execute_search":"execute_search",
        "end":END
    }
)

# After we execute the search, we ALWAYS want to go back to the planner
# to let it see the results of the tool call. This creates our agentic loop!
graph_builder.add_edge("execute_search", "planner")

graph = graph_builder.compile()

Start at planner.

After planner, the route_after_planner function runs.

If a tool is needed, it sends us to execute_search.

After execute_search, the standard edge (add_edge) sends us back to planner, creating a cycle. The planner now sees the tool's results in the message history and can decide what to do next.

If no tool is needed, the router sends us to END, and the graph finishes.